## TOP

In [1]:
from glob import glob
import pandas as pd

In [2]:
dataset = 'E-Microbial'
OUTPUT_DIRECTORY = 'output/set1_subset2_products'
restricted = True

In [3]:
gut_ECs = set(pd.read_csv('sup_data/supplementary_data_2.csv')['EC_Number'].to_list())

In [4]:
dataset_df = pd.read_excel('sup_data/supplementary_data_1.xlsx', sheet_name=dataset)
dataset_df.head()

,DrugName,DrugID,DrugSmiles,DrugInchi,DrugInchiKey,EnzymeName,ECNumber,ProductName,ProductID,ProductSmiles,ProductInchi,ProductInchiKey,Microbe,ProductRecalled,EnzymeRecalled,BothRecalled,OtherEnzymes,FractionInGut
0,Balsalazide,D07488,OC(=O)CCNC(=O)c1ccc(cc1)\N=N\c1ccc(O)c(c1)C(O)=O,InChI=1S/C17H15N3O6/c21-14-6-5-12(9-13(14)17(2...,IPOKCKJONYRRHP-FMQUCBEESA-N,Azoreductase,1.6.5.-,5-aminosalicyclic acid,CID4075,C1=CC(=C(C=C1N)C(=O)O)O,InChI=1S/C7H7NO3/c8-4-1-2-6(9)5(3-4)7(10)11/h1...,KBOPZPXVLCULAV-UHFFFAOYSA-M,NoMicrobeRecorded,True,False,False,1.1.1.-; 1.14.12.17; 6.2.1.3; 2.3.1.40; 6.2.1....,0.130
1,Clonazepam,D00280,C1C(=O)NC2=C(C=C(C=C2)[N+](=O)[O-])C(=N1)C3=CC...,InChI=1S/C15H10ClN3O3/c16-12-4-2-1-3-10(12)15-...,DGBIGWXXNGSACT-UHFFFAOYSA-N,Nitroreductase,1.7.1.16,7-aminoclonazepam,CID188298,C1C(=O)NC2=C(C=C(C=C2)N)C(=N1)C3=CC=CC=C3Cl,InChI=1S/C15H12ClN3O/c16-12-4-2-1-3-10(12)15-1...,HEFRPWRJTGLSSV-UHFFFAOYSA-N,NoMicrobeRecorded,True,True,True,4.1.1.98; 4.1.1.98; 4.1.1.63; 4.1.1.63; 2.5.1....,0.130
2,Digoxin,D00298,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,InChI=1S/C41H64O14/c1-19-36(47)28(42)15-34(50-...,LTMHDMANZUZIPE-UHFFFAOYSA-N,Cardiac glycoside reductase,1.3.2.-,Dihydrodigoxin,CID92217,CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5...,InChI=1S/C41H66O14/c1-19-36(47)28(42)15-34(50-...,QYVJGQUFXQMOOE-UHFFFAOYSA-N,E. lenta,False,False,False,2.4.1.325; 2.7.1.25; 3.1.3.7; 4.2.1.126; 2.4.1...,0.139
3,Genistin,C09126,C1=CC(=CC=C1C2=COC3=CC(=CC(=C3C2=O)O)OC4C(C(C(...,InChI=1S/C21H20O10/c22-7-15-18(26)19(27)20(28)...,ZCOLJUOHXJRHDI-UHFFFAOYSA-N,β-glucosidase,3.2.1.21,Geinstein,CID5280961,C1=CC(=CC=C1C2=COC3=CC(=CC(=C3C2=O)O)O)O,InChI=1S/C15H10O5/c16-9-3-1-8(2-4-9)11-7-20-13...,TZBJGXHYKVUXJN-UHFFFAOYSA-N,NoMicrobeRecorded,True,True,True,2.7.1.2; 3.1.3.68; 2.7.1.55; 2.7.1.1; 2.1.1.11...,0.113
4,Irinotecan,D08086,CCC1=C2CN3C(=CC4=C(C3=O)COC(=O)C4(CC)O)C2=NC5=...,InChI=1S/C33H38N4O6/c1-3-22-23-16-21(43-32(40)...,UWKQSNNFCGGAFS-UHFFFAOYSA-N,β-glucuronidase,3.2.1.31,SN-38,CID104842,CCC1=C2CN3C(=CC4=C(C3=O)COC(=O)C4(CC)O)C2=NC5=...,InChI=1S/C22H20N2O5/c1-3-12-13-7-11(25)5-6-17(...,FJHBVJOVLFPMQE-QFIPXVFZSA-N,NoMicrobeRecorded,True,False,False,1.3.1.75; 1.3.7.13; 1.14.13.-; 1.14.15.3; 1.14...,0.214


In [5]:
drug_ids = dataset_df['DrugID'].to_list()

expected_product_full_inchi_keys = dataset_df['ProductInchiKey'].to_list()
expected_product_inchi_keys = [inchi_key.split('-')[0] for inchi_key in expected_product_full_inchi_keys]

if dataset in ['E-Microbial', 'E-Biological']: # if there is enzyme information to process
    enzymes_present = True
    expected_ecs = dataset_df['ECNumber'].to_list()
else: enzymes_present = False

In [6]:
def check_ecs(ec1, ec2):
    ec1_list = ec1.split('.')
    ec2_list = ec2.split('.')
    
    if len(ec1_list) != 4 or len(ec2_list) != 4: return False
    
    if not ec1_list[0] == ec2_list[0]: return False
    elif not ec1_list[1] == ec2_list[1]: return False
    elif not (ec1_list[2] == ec2_list[2] or ec1_list[2] == '-' or ec2_list[2] == '-'): return False
    elif not (ec1_list[3] == ec2_list[3] or ec1_list[3] == '-' or ec2_list[3] == '-'): return False
    else: return True        

## Computing results

### Fraction in Gut

In [7]:
# how many of the predicted enzymes are in the gut?

ecs_seen = set() 

gt_drug_product_folder_paths = glob(f'{OUTPUT_DIRECTORY}/Met*')
for i in range(len(drug_ids)):
    drug_id = drug_ids[i]
    product = expected_product_inchi_keys[i]
    relevant_folder_paths = [path for path in gt_drug_product_folder_paths if drug_id in path]
    ec_worked = False
    if not len(relevant_folder_paths) == 0: 
        
        relevant_folder_path = relevant_folder_paths[0]
        
        summary_df = pd.read_csv(f"{relevant_folder_path}/{drug_id}_output_summary.csv")
        
        
        for ec in summary_df['EC']:
            if ec == 'NoEC': continue
            ecs_seen = set(list(ecs_seen) + [ec])

dataset_ecs_in_gut = []
            
for ec in list(ecs_seen):
    in_gut = False
    for ref_ec in gut_ECs:
        if check_ecs(ec, ref_ec):
            in_gut = True
            break
    if in_gut:
        dataset_ecs_in_gut.append(ec)
        
print(f'Pipeline run on {OUTPUT_DIRECTORY} gave us {len(dataset_ecs_in_gut)} enzymes in the gut out of a total of {len(ecs_seen)}')

Pipeline run on output/set1_subset2_products gave us 37 enzymes in the gut out of a total of 38


### Product Recall

In [8]:
# Product recall
correct_count = 0
total_count = 0 

gt_drug_product_folder_paths = glob(f'{OUTPUT_DIRECTORY}/Met*')
for i in range(len(drug_ids)):
    drug_id = drug_ids[i]
    product = expected_product_inchi_keys[i]
    relevant_folders = [path for path in gt_drug_product_folder_paths if drug_id in path]
    if len(relevant_folders) == 0: 
        print(drug_id)
        continue
    relevant_folder_path = relevant_folders[0]
    summary_df = pd.read_csv(f"{relevant_folder_path}/{drug_id}_output_summary.csv")
    
    if restricted:
        summary_df = summary_df.loc[summary_df['EC'].apply(lambda ec: ec in dataset_ecs_in_gut)]
    
    predicted_products = set([inchi_key.split('-')[0] for inchi_key in summary_df['ProductInChiKey'].to_list() if isinstance(inchi_key, str)])
    product_found = product in predicted_products
    print_message = {True: 'DID!', False: 'did NOT...'}[product_found]
    print(f"{drug_id} is predicted to produce {product} which it {print_message}")
    if product_found: correct_count += 1
    total_count += 1
    
print(f'Pipeline run on {OUTPUT_DIRECTORY} was able to correctly recall {correct_count} out of {len(drug_ids)} products')

D07488 is predicted to produce KBOPZPXVLCULAV which it DID!
D00280 is predicted to produce HEFRPWRJTGLSSV which it DID!
D00298 is predicted to produce QYVJGQUFXQMOOE which it did NOT...
C09126 is predicted to produce TZBJGXHYKVUXJN which it DID!
D08086
C05623 is predicted to produce CFFZDZCDUFSOFZ which it did NOT...
C05623 is predicted to produce FERIUCNNQQJTOY which it did NOT...
C05623 is predicted to produce QTBSBXVTEAMEQO which it did NOT...
C09789 is predicted to produce FTVWIRXFELQLPI which it DID!
CID8625 is predicted to produce FDDDEECHVMSUSB which it DID!
D00531 is predicted to produce OYOUQHVDCKOOAL which it DID!
D08295 is predicted to produce KBOPZPXVLCULAV which it DID!
D08542 is predicted to produce FDDDEECHVMSUSB which it DID!
D00448 is predicted to produce GECHUMIMRBOMGK which it DID!
D00448 is predicted to produce KBOPZPXVLCULAV which it DID!
D00059 is predicted to produce VYFYYTLLBUKUHU which it DID!
D00584
C17457 is predicted to produce WPYMKLBDIGXBTP which it did NO

### EC Recall

In [9]:
# Enzyme recall
if not enzymes_present:
    print('The dataset under analysis does not contain enzyme information')
else:
    correct_count = 0
    total_count = 0 

    gt_drug_product_folder_paths = glob(f'{OUTPUT_DIRECTORY}/Met*')
    for i in range(len(drug_ids)):
        drug_id = drug_ids[i]
        product = expected_product_inchi_keys[i]
        current_ec = expected_ecs[i]
        relevant_folder_paths = [path for path in gt_drug_product_folder_paths if drug_id in path]
        ec_worked = False
        if not len(relevant_folder_paths) == 0: 
            
            relevant_folder_path = relevant_folder_paths[0]
            
            summary_df = pd.read_csv(f"{relevant_folder_path}/{drug_id}_output_summary.csv")
            
            
            if restricted:
                summary_df = summary_df.loc[summary_df['EC'].apply(lambda ec: ec in dataset_ecs_in_gut)]
            
            
            for ec in summary_df['EC']:
                if check_ecs(current_ec, ec):
                    print(f"{drug_id} was supposed to react with {current_ec} and it was predicted to react with {ec}")
                    ec_worked = True
                    correct_count += 1
                    break
        if not ec_worked: print(f"Enzyme for {drug_id} not retrieved...")
        total_count += 1
        
    print(f'Pipeline run on {OUTPUT_DIRECTORY} was able to correctly recall {correct_count} out of {len(drug_ids)} enzymes')

Enzyme for D07488 not retrieved...
D00280 was supposed to react with 1.7.1.16 and it was predicted to react with 1.7.1.-
Enzyme for D00298 not retrieved...
C09126 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.21
Enzyme for D08086 not retrieved...
C05623 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.21
C05623 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.21
C05623 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.21
C09789 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.21
Enzyme for CID8625 not retrieved...
D00531 was supposed to react with 1.7.1.16 and it was predicted to react with 1.7.1.-
Enzyme for D08295 not retrieved...
Enzyme for D08542 not retrieved...
Enzyme for D00448 not retrieved...
Enzyme for D00448 not retrieved...
D00059 was supposed to react with 4.1.1.25 and it was predicted to react with 4.1.1.25
Enzyme for D00584 not ret

### Both Recall

In [10]:
# Both recall
if not enzymes_present:
    print('The dataset under analysis does not contain enzyme information')
else:
    correct_count = 0
    total_count = 0 

    gt_drug_product_folder_paths = glob(f'{OUTPUT_DIRECTORY}/Met*')
    for i in range(len(drug_ids)):
        drug_id = drug_ids[i]
        product = expected_product_inchi_keys[i]
        current_ec = expected_ecs[i]
        relevant_folders = [path for path in gt_drug_product_folder_paths if drug_id in path]
        if len(relevant_folders) == 0: continue
        relevant_folder_path = relevant_folders[0]
        summary_df = pd.read_csv(f"{relevant_folder_path}/{drug_id}_output_summary.csv")
        
        if restricted:
            summary_df = summary_df.loc[summary_df['EC'].apply(lambda ec: ec in dataset_ecs_in_gut)]
        
        predicted_products = set([inchi_key.split('-')[0] for inchi_key in summary_df['ProductInChiKey'].to_list() if isinstance(inchi_key, str)])
        product_found = product in predicted_products
        
        if not product_found: 
            print(f"{drug_id} not relevant")
            continue
        relevant_rows = summary_df.loc[summary_df['ProductInChiKey'].apply(lambda key: key.split('-')[0] == product if isinstance(key, str) else False)]
        ec_worked = False
        
        if len(relevant_rows) > 0:
            
            for ec in relevant_rows['EC']:
                if check_ecs(current_ec, ec):
                    print(f"{drug_id} was supposed to react with {current_ec} and it was predicted to react with {ec}")
                    ec_worked = True
                    correct_count += 1
                    break
        if not ec_worked: print(f"Enzyme for {drug_id} not retrieved...")
        total_count += 1
            
    print(f'Pipeline run on {OUTPUT_DIRECTORY} was able to SIMULTANEOUSLY recall {correct_count} out of {len(drug_ids)} cases') 

Enzyme for D07488 not retrieved...
D00280 was supposed to react with 1.7.1.16 and it was predicted to react with 1.7.1.-
D00298 not relevant
C09126 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.-
C05623 not relevant
C05623 not relevant
C05623 not relevant
C09789 was supposed to react with 3.2.1.21 and it was predicted to react with 3.2.1.-
Enzyme for CID8625 not retrieved...
D00531 was supposed to react with 1.7.1.16 and it was predicted to react with 1.7.1.-
Enzyme for D08295 not retrieved...
Enzyme for D08542 not retrieved...
Enzyme for D00448 not retrieved...
Enzyme for D00448 not retrieved...
D00059 was supposed to react with 4.1.1.25 and it was predicted to react with 4.1.1.25
C17457 not relevant
C01197 not relevant
D00157 not relevant
D00120 not relevant
D00104 not relevant
D01230 not relevant
Pipeline run on output/set1_subset2_products was able to SIMULTANEOUSLY recall 5 out of 23 cases
